In [8]:
from flask import Flask, render_template, request, redirect

app = Flask(__name__, template_folder='C:/Users/Ashlesha Baral/Documents')

def check_user(email, name, password):
    # Open the CSV file and check if the email exists
    with open('D:\\code\\Users.csv', 'r') as file:
        for line in file:
            user_data = line.strip().split(',')
            if user_data[0] == email:
                # If email matches, check if name and password match
                if user_data[1] == password and user_data[2] == name:
                    # Name and password match, return True
                    return True
                else:
                    # Name or password does not match, return False
                    return False
    # Email not found, return False
    return False

def add_user(name, email, password):
    # Open the CSV file and check if the email already exists
    with open('D:\\code\\Users.csv', 'r') as file:
        for line in file:
            user_data = line.strip().split(',')
            if user_data[0] == email:
                return False  # Email already exists
    
    # Email does not exist, so append the new user data to the CSV file
    try:
        with open('D:\\code\\Users.csv', 'a') as file:
            file.write(f"{email},{password},{name}\n")
        return True
    except Exception as e:
        print(f"Error occurred while writing to Users.csv: {str(e)}")
        return False

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        password = request.form['password']

        if check_user(email, name, password):
            # Redirect to the specified HTML page
            return redirect('/static/indexfinal.html')
        else:
            if add_user(name, email, password):
                # Redirect to the specified HTML page
                return redirect('/static/indexfinal.html')
            else:
                return 'Invalid user!'
    return render_template('mehesignin1.html')

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [6]:
from flask import Flask, render_template, request, redirect, url_for, flash
from textblob import TextBlob
import pandas as pd
import random

app = Flask(__name__, template_folder="C:/Users/Ashlesha Baral/Documents")

# Load your dataset containing questions
data = pd.read_csv("C:/Users/Ashlesha Baral/Documents/MH.csv", encoding='latin1')

# Function to perform sentiment analysis on a given text
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

# Function to assign sentiment scores dynamically based on sentiment analysis
def assign_sentiment_scores(question_sentiment, response_sentiment):
    # Determine overall sentiment score based on both question and response sentiment
    overall_sentiment_score = (question_sentiment + response_sentiment) / 2
    return overall_sentiment_score

# Function to generate mental health conclusion based on overall sentiment score
def generate_mental_health_conclusion(overall_sentiment, additional_symptoms):
    conclusion = ""
    if overall_sentiment >= 0.5:
        conclusion += "You have a positive outlook on your mental health."
    elif overall_sentiment >= 0:
        conclusion += "Your mental health seems stable."
    elif overall_sentiment >= -0.5:
        conclusion += "You may be experiencing some challenges with your mental health."
    else:
        conclusion += "You may need to seek professional help for your mental health."

    if additional_symptoms:
        conclusion += f" Additional symptoms: {additional_symptoms}"
    
    return conclusion

# Route for the survey form
@app.route('/', methods=['GET', 'POST'])
def survey():
    if request.method == 'POST':
        sentiment_scores = []
        for i in range(10):
            question_idx = random.randint(0, len(data) - 1)
            question = data.iloc[question_idx, 0]  # Get random question from dataset
            response = request.form.get(f"response_{i}", "")  # Get user response from form
            question_sentiment = analyze_sentiment(question)
            response_sentiment = analyze_sentiment(response)
            sentiment_score = assign_sentiment_scores(question_sentiment, response_sentiment)
            sentiment_scores.append(sentiment_score)

        if len(sentiment_scores) == 10:
            additional_symptoms = request.form.get('additional_symptoms', '')  # Get additional symptoms from form
            overall_sentiment = sum(sentiment_scores) / len(sentiment_scores)
            conclusion = generate_mental_health_conclusion(overall_sentiment, additional_symptoms)
            flash(conclusion)
            return redirect(url_for('survey'))
        else:
            flash("Please answer all 10 questions before generating the report.")
            return redirect(url_for('survey'))
    questions = data.sample(n=10)  # Select random 10 questions
    text_question = data.iloc[0, 0]  # Get the text question
    questions_with_responses = []
    for idx, (_, row) in enumerate(questions.iterrows(), start=1):
        question_responses = [f"{idx}. {row[0]}"] + row[1:].tolist()  # Include serial number and options
        questions_with_responses.append(question_responses)
    return render_template('survey.html', questions_with_responses=questions_with_responses, text_question=text_question)

if __name__ == '__main__':
    app.run(debug=False,port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
C:\Users\Ashlesha Baral\AppData\Local\Temp\ipykernel_17812\1064717743.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  question_responses = [f"{idx}. {row[0]}"] + row[1:].tolist()  # Include serial number and options
127.0.0.1 - - [15/Apr/2024 20:22:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 20:22:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 20:22:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 20:22:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Apr/2024 20:22:27] "GET /favicon.ico HTTP/1.1" 404 -
